# ru_transformers - GPT-2 tuning and generation for horoscopes
Original git: https://github.com/mgrankin/ru_transformers

Открыть ноутбук можно на Colab:
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1ZnTI4Cg9YGK4wIdE16AjdfiUpcxTpa0A?usp=sharing)

In [1]:
import regex as re
import os
from fastai.basics import *
from multiprocessing import Pool
#from colab_ssh import launch_ssh


In [2]:
%tensorflow_version 2.x
!rm -rf '/content/ru_transformers' || :
!git clone https://github.com/mgrankin/ru_transformers
%cd ru_transformers
from google.colab import drive
drive.mount('/content/gdrive') # для сохранения моделей и импорта данных
!pip install -q -r "tpu_requirements.txt"
!pip install -q tendo
!pip install awscli
!pip install colab_ssh
!nvidia-smi

Cloning into 'ru_transformers'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 1959 (delta 14), reused 22 (delta 6), pack-reused 1921
Receiving objects: 100% (1959/1959), 5.72 MiB | 11.56 MiB/s, done.
Resolving deltas: 100% (1292/1292), done.
/content/ru_transformers
Mounted at /content/gdrive
     |████████████████████████████████| 3.5MB 4.6MB/s 
     |████████████████████████████████| 3.8MB 41.7MB/s 
     |████████████████████████████████| 368kB 57.6MB/s 
     |████████████████████████████████| 1.1MB 55.9MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
     |████████████████████████████████| 1.7MB 59.7MB/s 
     |████████████████████████████████| 133kB 60.7MB/s 
     |████████████████████████████████| 890kB 59.8MB/s 
     |████████████████████████████████| 61kB 9.2MB/s 
     |████████████████████████████████| 9.2MB 33.9MB/s 
     |████████████████████████████████| 71kB 10

Fri Oct 30 16:21:38 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |     10MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
%set_env CUDA_HOME=/usr/local/cuda-10.1
!git clone https://github.com/NVIDIA/apex
%cd apex
# fix setup.py if complains for version mismatch
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
%cd ..

env: CUDA_HOME=/usr/local/cuda-10.1
Cloning into 'apex'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 7456 (delta 0), reused 0 (delta 0), pack-reused 7455
Receiving objects: 100% (7456/7456), 13.91 MiB | 28.95 MiB/s, done.
Resolving deltas: 100% (5038/5038), done.
/content/ru_transformers/apex
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-icp73ohj
Created temporary directory: /tmp/pip-req-tracker-tyy7lu5r
Created requirements tracker '/tmp/pip-req-tracker-tyy7lu5r'
Created temporary directory: /tmp/pip-install-qukjdlre
Processing /content/ru_transformers/apex
  Created temporary directory: /tmp/pip-req-build-dtsk5qsd
  Added file:///content/ru_transformers/apex to build tracker '/tmp/pi

In [4]:
!cd ..; aws s3 sync --no-sign-request s3://models.dobro.ai/gpt2/ru/unfreeze_all gpt2 #загрузка моделей

download: s3://models.dobro.ai/gpt2/ru/unfreeze_all/m_checkpoint-3364613/config.json to gpt2/m_checkpoint-3364613/config.json
download: s3://models.dobro.ai/gpt2/ru/unfreeze_all/m_checkpoint-3364613/step.txt to gpt2/m_checkpoint-3364613/step.txt
download: s3://models.dobro.ai/gpt2/ru/unfreeze_all/s_checkpoint-1900000/step.txt to gpt2/s_checkpoint-1900000/step.txt
download: s3://models.dobro.ai/gpt2/ru/unfreeze_all/m_checkpoint-3364613/training_args.bin to gpt2/m_checkpoint-3364613/training_args.bin
download: s3://models.dobro.ai/gpt2/ru/unfreeze_all/s_checkpoint-1900000/config.json to gpt2/s_checkpoint-1900000/config.json
download: s3://models.dobro.ai/gpt2/ru/unfreeze_all/s_checkpoint-1900000/training_args.bin to gpt2/s_checkpoint-1900000/training_args.bin
download: s3://models.dobro.ai/gpt2/ru/unfreeze_all/m_checkpoint-3364613/encoder.model to gpt2/m_checkpoint-3364613/encoder.model
download: s3://models.dobro.ai/gpt2/ru/unfreeze_all/s_checkpoint-1900000/encoder.model to gpt2/s_check

In [6]:
DATASET_PATH = "/content/gdrive/My Drive/MADE_graduation_project_ML_horoscopes/dataset"
SAVE_TO_PATH = "./dataset/prepared/"

In [7]:
def process_function(path_to_file):
    match = re.compile(r'(?=[^ ])([\W])([\w])')
    match2 = re.compile('(.|\s)\\1\\1+')
    with open(path_to_file, 'r') as f:
        lines = f.read()
    if lines and lines[0] != ' ': lines = ' ' + lines
    lines = match.sub(r'\g<1> \g<2>', lines)
    lines = match2.sub(r'\1'*3, lines)
    path = os.path.join(SAVE_TO_PATH, os.path.split(path_to_file)[1])
    with open(path, 'w') as handle:
        handle.write(lines)

In [8]:
txts = get_files(DATASET_PATH, '.txt')
if not os.path.exists(SAVE_TO_PATH):
    os.makedirs(SAVE_TO_PATH)


for _ in progress_bar(Pool(64).imap_unordered(process_function, txts), len(txts)):
    pass

In [9]:
!mkdir ./dataset/validation

In [10]:
!mkdir ./dataset/train

In [11]:
!mv ./dataset/prepared/train.txt ./dataset/train/


In [12]:
!mv ./dataset/prepared/validation.txt ./dataset/validation

In [13]:
os.chdir('/content/')

In [14]:
%set_env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [ ]:
# Можно и по ssh root@some.domen -p 12345
#launch_ssh("YOUR TOKEN HERE", "YOUR PASSWORD HERE")

In [15]:
os.chdir('/content/ru_transformers')

In [16]:
!python3 run_lm_finetuning.py \
    --output_dir="/content/gdrive/My Drive/MADE_graduation_project_ML_horoscopes/ruGPT2_mail_corpus" \
    --model_type=gpt2 \
    --model_name_or_path=gpt2/s_checkpoint-1900000/ \
    --do_train \
    --train_data_file=./dataset/train/train.txt \
    --per_gpu_train_batch_size=2 \
    --save_steps=10000 \
    --logging_steps=1 \
    --warmup_samples 16000 \
    --learning_rate 3e-5 \
    --tokenizer_class YTEncoder \
    --tokenizer_name bpe/yt.model \
    --do_eval \
    --evaluate_during_training \
    --eval_steps 1000 \
    --eval_data_file=./dataset/validation/validation.txt \
    --num_train_epochs 5.0 \
    --unfreeze_level 0 \
    --fp16 \
    --fp16_opt_level O2 \
    --overwrite_output_dir

2020-10-30 16:35:56.982496: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-30 16:35:58.169016: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-10-30 16:35:58.169431: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-10-30 16:35:58.170110: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2020-10-30 16:35:58.170147: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-30 16:35:58.414543: I tensorflow/stream_executor/platform/default

In [17]:
!python3 run_lm_finetuning.py \
    --output_dir="/content/gdrive/My Drive/MADE_graduation_project_ML_horoscopes/ruGPT2_mail_corpus_more_eps" \
    --model_type=gpt2 \
    --model_name_or_path=gpt2/s_checkpoint-1900000/ \
    --do_train \
    --train_data_file=./dataset/train/train.txt \
    --per_gpu_train_batch_size=2 \
    --save_steps=10000 \
    --logging_steps=1 \
    --warmup_samples 16000 \
    --learning_rate 3e-5 \
    --tokenizer_class YTEncoder \
    --tokenizer_name bpe/yt.model \
    --do_eval \
    --evaluate_during_training \
    --eval_steps 1000 \
    --eval_data_file=./dataset/validation/validation.txt \
    --num_train_epochs 9.0 \
    --unfreeze_level 0 \
    --fp16 \
    --fp16_opt_level O2 \
    --overwrite_output_dir

2020-10-30 16:45:51.445191: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-30 16:45:52.591813: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-10-30 16:45:52.591983: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-10-30 16:45:52.592597: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2020-10-30 16:45:52.592632: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-30 16:45:52.594947: I tensorflow/stream_executor/platform/default

In [18]:
from transformers import GPT2Tokenizer,GPT2LMHeadModel
from run_generation import sample_sequence
from yt_encoder import YTEncoder
from ipywidgets import interact, interact_manual

import pandas as pd
import ipywidgets as widgets

In [24]:
contexts = ['Насыщенный интересный день. За любые дела',
            'Будьте практичны. Этот день может',
            'Благоприятный день. Если вы проявите инициативу',
            'Ради достижения поставленных целей вы готовы',
            'День пройдет не совсем так, как вы ожидали, но огорчаться',
            'Вы многого добьетесь, если не станете',
            'День может принести', 
            'Благоприятный день для общения. Вы отлично',
            'Спокойный день, благоприятный и плодотворный. Он подходит для',
            'Удачный день. Беспокоиться особенно не о чем,',
            'Вы сделаете много полезного и для себя,'
            'Старайтесь не нервничать. Этот день'
            ]

In [25]:
models = ["/content/gdrive/My Drive/MADE_graduation_project_ML_horoscopes/ruGPT2_mail_corpus_more_eps/checkpoint-1901611",'/content/gdrive/My Drive/MADE_graduation_project_ML_horoscopes/ruGPT2_mail_corpus/checkpoint-1900895' ]

In [52]:
full_results = []

In [39]:
contexts = ['Насыщенный интересный день. За любые дела',
            'Будьте практичны. Этот день может',
            'Благоприятный день. Если вы проявите инициативу',
            'Ради достижения поставленных целей вы готовы',
            'День пройдет не совсем так, как вы ожидали, но огорчаться',
            'Вы многого добьетесь, если не станете',
            'День может принести', 
            'Благоприятный день для общения. Вы отлично',
            'Спокойный день, благоприятный и плодотворный. Он подходит для',
            'Удачный день. Беспокоиться особенно не о чем,',
            'Вы сделаете много полезного и для себя,'
            'Старайтесь не нервничать. Этот день'
            ]

In [53]:
for model_name in models:
  tokenizer = YTEncoder.from_pretrained(model_name)
  model = GPT2LMHeadModel.from_pretrained(model_name)
  results = []
  num_return = 3
  for input_context in contexts:
    input_ids = tokenizer.encode(input_context)
    outputs = sample_sequence(model, 40, tokenizer.encode(input_context), 3, 0.6)
    for i in range(num_return):
      results.append(tokenizer.decode(outputs[i]).replace('\n',' ').replace('\t',' '))
  full_results.append(results)

10/30/2020 17:50:48 - INFO - transformers.configuration_utils -   loading configuration file /content/gdrive/My Drive/MADE_graduation_project_ML_horoscopes/ruGPT2_mail_corpus_more_eps/checkpoint-1901611/config.json
10/30/2020 17:50:48 - INFO - transformers.configuration_utils -   Model config {
  "attn_pdrop": 0.1,
  "embd_pdrop": 0.1,
  "finetuning_task": null,
  "initializer_range": 0.02,
  "is_decoder": false,
  "layer_norm_epsilon": 1e-05,
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 12,
  "n_positions": 1024,
  "num_labels": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torchscript": false,
  "use_bfloat16": false,
  "vocab_size": 50257
}

10/30/2020 17:50:48 - INFO - transformers.modeling_utils -   loading weights file /c

In [54]:
results_df = pd.DataFrame(full_results)

In [55]:
results_df = results_df.transpose()

In [56]:
results_df.to_csv('tuned_GPT2_generation_results_short.csv')